In [ ]:
from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking
from implicit.lmf import LogisticMatrixFactorization
from implicit.approximate_als import NMSLibAlternatingLeastSquares,AnnoyAlternatingLeastSquares,FaissAlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender,CosineRecommender,TFIDFRecommender,BM25Recommender

from scipy.sparse import csr_matrix
import pandas as pd
import numpy as np

In [ ]:
# 定义样本集;
df = pd.DataFrame([['uid_001', 'iid_001', 0.2],
                   ['uid_002', 'iid_002', 0.5],
                   ['uid_003', 'iid_003', 0.7],
                   ['uid_004', 'iid_004', 0.8],
                   ['uid_005', 'iid_004', 0.5],
                   ['uid_006', 'iid_001', 0.6],
                   ['uid_007', 'iid_002', 0.7],
                   ['uid_008', 'iid_003', 0.9],
                   ['uid_009', 'iid_004', 0.1]], columns=['user_raw_id', 'item_raw_id', 'weight'])
# 对象编码(原始序号转内部序号);
df['user_id'], user_id_uniques = pd.factorize(values=df['user_raw_id'], sort=False, na_sentinel=-1, size_hint=None)
df['item_id'], item_id_uniques = pd.factorize(values=df['item_raw_id'], sort=False, na_sentinel=-1, size_hint=None)
# 调整顺序(演示专用);
df = df[['user_raw_id', 'item_raw_id', 'user_id', 'item_id', 'weight']]
df

In [ ]:
# 压缩稀疏行矩阵: 行为用户(user_id), 列为物品(item_id);
user_item_sparse_matrix = csr_matrix((df['weight'], (df['user_id'], df['item_id'])))
# 压缩稀疏列矩阵: 行为物品(item_id), 列为用户(user_id);
item_user_sparse_matrix = csr_matrix((df['weight'], (df['item_id'], df['user_id'])))
# 交替最小二乘法;
als = AlternatingLeastSquares(
    factors=100,
    regularization=0.01,
    dtype=np.float32,
    use_native=True,
    use_cg=True,
    use_gpu=False,
    iterations=15,
    calculate_training_loss=False,
    num_threads=0,
    random_state=42
)
als.fit(item_users=item_user_sparse_matrix, show_progress=False)
# 相关属性;
als.item_factors

In [ ]:
als.user_factors
